In [1]:
import pandas as pd
import os
import sys
import json

In [6]:
if sys.platform.startswith("linux"):
    data_dir = "/bask/projects/v/vjgo8416-amber/data/gbif_download_standalone/"
elif sys.platform == "darwin":
    data_dir = "/Users/lbokeria/Documents/projects/gbif-species-trainer-data/"
else:
    print("Not linux or mac!")

checklist_name = "uksi-moths-keys-nodup"

df = pd.read_csv(os.path.join("../species_checklists",checklist_name+".csv"))

In [7]:
df["n_imgs"] = ""
for idx, row in df.iterrows():
    
    family  = row["family_name"]
    genus   = row["genus_name"]
    species = row["gbif_species_name"]
    
    # Check if directory exists 
    species_dir = (
        os.path.join(data_dir,"gbif_images",family,genus,species)
        )
    
    if os.path.isdir(species_dir):
        n_images_on_disk = len(
            [f for f in os.listdir(species_dir) if f.lower().endswith('.jpg') ]
            )
        # print(f"{species} Count files method has", n_images_on_disk, "images")
        
        # Load metadata
        try:
            
            with open(os.path.join(species_dir,"meta_data.json")) as file:
            
                meta_data = json.load(file)
            
            # Count the number of images for this species
            n_imgs_metadata = 0

            for key, value in meta_data.items():
                if value.get("image_is_downloaded") == True:
                    n_imgs_metadata += 1
            # print(f"{species} Count json metadata has", n_imgs_metadata, "images") 
            
            try:
                # 2nd way of counting images
                md2 = pd.read_json(
                    os.path.join(species_dir,"meta_data.json"), orient = 'index'
                    )
                
                if md2.empty:
                    md2_n_imgs_downloaded = 0
                else:
                    md2_n_imgs_downloaded = md2["image_is_downloaded"].sum()

                # print(f"{species} Count dataframe metadata has", md2_n_imgs_downloaded, "images")            

                if md2_n_imgs_downloaded != n_imgs_metadata:
                    print(f"Two ways of calculating metadata images don't match for {species_dir}")
            except Exception as e:
                print(e)
                print(f"{species} error counting dataframe way: {e}")
                        
            # Do n images match?
            if n_images_on_disk == n_imgs_metadata:
                # print(f"N images match for {species_dir}")
                pass
            else:
                print(
                    f"N images MISmatch. File count is {n_images_on_disk}, "
                    f"metadata has {n_imgs_metadata}, species: {species}"
                )
        except Exception as e:
            pass
            print(f"No metadata for {species_dir}")
            
    else:
        
        n_images_on_disk = 0
    
    # print(f"{species} has {n_images_on_disk} images")
    
    # Record this
    df.loc[idx, "n_imgs"] = n_images_on_disk
    
    

In [8]:
df

,accepted_taxon_key,order_name,family_name,genus_name,species_name_provided,authority_name_provided,search_species_name,gbif_species_name,confidence,status,match_type,rank,source,n_imgs
0,1860979,Lepidoptera,Adelidae,Adela,Adela croesella,"Scopoli, 1763",Adela croesella,Adela croesella,99,ACCEPTED,EXACT,SPECIES,London06October2023,350
1,1860983,Lepidoptera,Adelidae,Adela,Adela cuprella,"Denis & Schiffermüller, 1775",Adela cuprella,Adela cuprella,98,ACCEPTED,EXACT,SPECIES,London06October2023,117
2,1860987,Lepidoptera,Adelidae,Adela,Adela reaumurella,"Linnaeus, 1758",Adela reaumurella,Adela viridella,99,SYNONYM,EXACT,SPECIES,London06October2023,1000
3,4522891,Lepidoptera,Adelidae,Cauchas,Cauchas fibulella,"Denis & Schiffermüller, 1775",Cauchas fibulella,Cauchas fibulella,99,ACCEPTED,EXACT,SPECIES,London06October2023,162
4,4522886,Lepidoptera,Adelidae,Cauchas,Cauchas rufimitrella,"Scopoli, 1763",Cauchas rufimitrella,Cauchas rufimitrella,99,ACCEPTED,EXACT,SPECIES,London06October2023,569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2656,7410701,Lepidoptera,Zygaenidae,Zygaena,Zygaena purpuralis,"Brünnich, 1763",Zygaena purpuralis,Zygaena purpuralis,99,ACCEPTED,EXACT,SPECIES,London06October2023,414
2657,7761138,Lepidoptera,Zygaenidae,Zygaena,Zygaena trifolii,"Esper, 1783",Zygaena trifolii,Zygaena trifolii,99,ACCEPTED,EXACT,SPECIES,London06October2023,900
2658,5107644,Lepidoptera,Zygaenidae,Zygaena,Zygaena viciae,"Denis & Schiffermüller, 1775",Zygaena viciae,Zygaena viciae,99,ACCEPTED,EXACT,SPECIES,London06October2023,1000
2659,9400895,Lepidoptera,Nepticulidae,Etainia,Etainia sericopeza,"Zeller, 1839",Etainia sericopeza,Etainia sericopeza,98,ACCEPTED,EXACT,SPECIES,London06October2023,100


In [ ]:
species_dir = "/bask/homes/r/rybf4168/vjgo8416-amber/data/gbif_download_standalone/gbif_images/tests/Adelidae/Nemophora/Nemophora aurisparsella"

with open(os.path.join(species_dir,"meta_data.json")) as file:

    meta_data = json.load(file)

meta_data

In [ ]:
a = pd.read_json(os.path.join(species_dir,"meta_data.json"), orient='index')
a.empty

In [ ]:
# Save the df
df.to_csv(os.path.join("../data_stats_files/","data_stats_"+checklist_name+".csv"))